## PFFP Code Intro

The purpose of this notebook is to detail the features and steps to process PFFP binary code information.

Date: 09/09/2024

Name: Jonathan Moore

## Library/Module importing

This step is so that the libraries required can be accessed by the local code

In [ ]:
import matplotlib.pyplot as plt
import sys
import os
import numpy as np

#### Add the BlueDrop Analysis library to the current path
This is necessary for the time being since the library hasn't been converted a python package yet

In [ ]:
sys.path.append("../../BlueDrop_Analysis_Lib")

### Local Imports
These are imports from the actual BlueDrop Analysis Lib that our team wrote

In [ ]:
# Local loads

# This class represent the Folder that contains the pffp data
from lib.data_classes.pffpFolder import pffpDataFolder

### Set the necessary paths
These paths will be used to retrieve the data, dimensions of the BlueDrop, and Calibration Data


In [ ]:
# Data folder
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops"
folder_dir = os.path.join(os.pardir, "Artic_exmp_data")

# PFFP sensor data
calibration_dir = os.path.join(os.pardir, "calibration_factors/BlueDrop_Calibration_Factors.xlsx")

# PFFP tip information
tip_dir = os.path.join(os.pardir, "calibration_factors/BluedDrop_Cone_Values.xlsx")


### Creating the pffp Data folder

The purpose of the next cell is to create the ```pffp_data_folder``` and load the correct calibration factors to convert the voltages measured by the BlueDrop sensors to engineering units (g's, meters, pressure (kPa))

In [ ]:
# Set the name for the survey 
name = "VIM_Mouth_1"

# Set the id for the pffp
id = 1

# Create the object that will represent the folder with the pffp data in it
pffp_data_folder = pffpDataFolder(folder_dir, pffp_id = id, calibration_factor_dir=calibration_dir, survey_name = name)

# Read the calibration data
pffp_data_folder.read_calibration_excel_sheet()

# Select the specific calibration params
if id == 3:
    pffp_data_folder.get_sensor_calibration_params("Sequim_2021")
elif id == 1:
    pffp_data_folder.get_sensor_calibration_params("March_2023")
elif id ==9:
    pffp_data_folder.get_sensor_calibration_params("October_2023")
else:
    raise ValueError("not a valid pffp id")


#### Display the selected calibration factors

In [ ]:
display(pffp_data_folder.calibration_excel_sheet)

### Find all the pffp files

The next cell finds all the pffp files and stores a reference to them. The way it does this is by searching to find all the ```.bin``` files in the folder. As such, make sure there are no other files in the folder that have a ```.bin``` file extension.

In [ ]:
# Init a pffpFile instances for each binary file and store the results in the folder
pffp_data_folder.store_pffp_files(recursive = False)

# Print Meta-data about the pffp_data_folder including the number of .bin files found.
print(pffp_data_folder) # Information about the folder

In [ ]:
# Figures out if the .bin file has a drop
pffp_data_folder.analyze_all_files(store_df=True, use_pore_pressure=False, 
                              select_accel=["2g_accel", "18g_accel", "50g_accel", "250g_accel"],
                              debug = False)

### Processing Drop Files

The next cell processes the files that unequivocally have drops in them.

This means that for each drop in each file that contains a drop the following are done:

* An object is created to represent each drop 
* The start and end of the drop are found
* The acceleration data is integrated to get the velocity and displacement

In [ ]:
# Figure out the beginning and the end of the drops if the files contain drops
pffp_data_folder.process_drop_files()

### Processing Funky Files

For funky files there's at least one drop in the file that the code isn't sure if it's a drop or not. As such, those files need to be processed by hand. The following cell details the steps that are necessary to do that.

If the code doesn't find any funky files that this step isn't necessary.

In [ ]:
# Steps to process funky files

# Loop over all the files that are funky
for file in pffp_data_folder.pffp_funky_files:

    # Manual Process the drop. This method will take you through the process of manually processing the drop
    file.manually_process_drops(interactive_plot=True)

### Print out folder metadata

Printing out the folder meta-data again to see how it's been updated.

In [ ]:
# Print information about the folder object
print(pffp_data_folder)

## Info on the next section

The following cells walk using the prebuilt plotting functions for files and drops. For simplicity instead referencing an element of the ```pffp_data_folder.pffp_drop_files``` list (eg. ```pffp_data_folder.pffp_drop_files[0]```) the file object will be stored in a variable named ```file```. Similary, instead of referring to a drop inside of ```pffp_data_folder.pffp_drop_files[i].drops[j]``` where ```i``` can range from 0 to the number of drop files minus one and ```j``` can range from 0 to the number of drops inside that specific file, the drop object will be stored in a variable named ```drop```. 

### Exploring the pffpFile object

In [ ]:
# Store the first drop file

# The load order is different on windows and linux. For the purpose of this demonstration make sure the same file and drop is selected

# Set the indices for the file, drop indices

if os.name == "nt":
    # windows computer
    file_index, drop_index = 0, 0
elif os.name == "posix":
    # Linux computer
    # file_index, drop_index = -2, 0 
    file_index, drop_index = -2, 1 

file = pffp_data_folder.pffp_drop_files[file_index]

# Store the first drop
drop = pffp_data_folder.pffp_drop_files[file_index].drops[drop_index]


In [ ]:
print(file.drops[0])

In [ ]:
# file.manually_add_drop(peak_index = None, file_drop_index = 99, peak_info = None, pressure_check = False)


In [ ]:
# file.manually_process_drops(interactive_plot=True, figsize = (12,12))

In [ ]:
drop = file.drops[-1]

In [ ]:
print(file)

In [ ]:
print(file.drops[0])

In [ ]:
drop = file.drops[0]
print(drop)
drop.impulse_df

In [ ]:
file.plot_drop_impulses(figsize = [4,6], save_figs = False, hold = True, legend = True,
                        colors = ["black", "blue", "green", "orange", "purple", "brown"],
                        units = {"Time":"s", "accel":"g", "velocity":"m/s", "displacement":"cm"},
                        line_style = ["solid", "dashed"])